# Классификация отзывов к фильмам: 
пример бинарной классификации

## Загрузка набора данных IMDB

In [2]:
from keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)

17464789/17464789 [==============================] - 4s 0us/step


Аргумент num_words=10000 означает, что в обучающих данных будет сохранено 
только 10 000 слов

In [3]:
# получаем словарь слов из базы данных imdb
word_index = imdb.get_word_index()
# получаем, обратное представление словаря, отображающее индексы в слова
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# декодируем отзыв
decode_review = ' '.join([reverse_word_index.get(i-3, '?') for i in train_data[0]])

1641221/1641221 [==============================] - 1s 0us/step


## Кодирование последовательностей целых чисел в бинарную матрицу

In [4]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    # Создание матрицы с формой
    results = np.zeros(len(sequences), dimension)
    for i, sequence in enumerate(sequences):
        # Запись единицы в элемент с данным индексом
        results[i, sequence] = 1
    return results

In [5]:
#  векторизуем данные
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

TypeError: Cannot interpret '10000' as a data type

In [ ]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

## Создаем структуру нейросети

In [ ]:
from keras import models, layers
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# вариант использования нестандартных функций потерь, метрик и оптимизатора
from keras import losses, metrics, optimizers
model.compile(optimizer=optimizers.RMSprop(lr=0.001), 
              loss=losses.binary_crossentropy,metrics=[metrics.binary_accuracy])

## Проверяем решение

In [ ]:
# создаем проверочный набор
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [ ]:
# производим обучение
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(partial_x_train, partial_y_train, epochs=20, batch_size=512,
                    validation_data=(x_val, y_val))

формируем графики потерь на этапах обучения и проверки

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(history)+1)
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

формируем графики точности на этапе обучения и проверки

In [ ]:
plt.clf()
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Обучение новой модели с нуля

In [ ]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', 
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

In [ ]:
model.predict(x_test)